# Van Den Broeck Pocket Geometry

This notebook explores the Van Den Broeck warp drive modification.

**Reference**: Van Den Broeck, C. (1999). Class. Quantum Grav. 16, 3973.

## Key Innovation

Van Den Broeck proposed a "pocket" modification where:
- **External radius** $R_{ext}$ is very small (microscopic)
- **Internal volume** is very large
- Energy requirements scale with $R_{ext}$, not internal volume

This dramatically reduces the total energy needed for a given passenger volume.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from warpbubblesim.metrics import AlcubierreMetric, VanDenBroeckMetric
from warpbubblesim.gr import compute_energy_density, check_energy_conditions
from warpbubblesim.viz.fields2d import plot_energy_density, plot_field_2d

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12

## 1. The Metric

Van Den Broeck modifies the spatial metric with an expansion factor $B(r)$:

$$ds^2 = -dt^2 + B(r)^2[(dx - v_s f dt)^2 + dy^2 + dz^2]$$

where:
- $B \gg 1$ inside (large internal volume)
- $B = 1$ outside (normal external appearance)

In [ ]:
# Create Van Den Broeck metric
vdb = VanDenBroeckMetric(
    v0=1.0,
    R_ext=0.1,    # Small external radius
    R_int=1.0,    # Large internal radius
    B_int=10.0,   # Expansion factor
    sigma=8.0
)

info = vdb.info()
print(f"Metric: {info['name']}")
print(f"Citation: {info['citation']}")
print(f"Parameters: {info['parameters']}")

## 2. The Expansion Factor B(r)

In [ ]:
# Plot B function for different configurations
r = np.linspace(0, 0.5, 200)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Different B_int values
ax = axes[0]
for B_int in [2, 5, 10, 50]:
    metric = VanDenBroeckMetric(R_ext=0.1, R_int=1.0, B_int=B_int)
    B = [metric.B_function(ri) for ri in r]
    ax.plot(r, B, label=f'$B_{{int}} = {B_int}$', linewidth=2)

ax.set_xlabel('r')
ax.set_ylabel('B(r)')
ax.set_title('Expansion Factor vs $B_{int}$')
ax.legend()
ax.grid(True, alpha=0.3)

# Different R_ext values
ax = axes[1]
r = np.linspace(0, 0.3, 200)
for R_ext in [0.01, 0.05, 0.1, 0.2]:
    metric = VanDenBroeckMetric(R_ext=R_ext, R_int=1.0, B_int=10.0)
    B = [metric.B_function(ri) for ri in r]
    ax.plot(r, B, label=f'$R_{{ext}} = {R_ext}$', linewidth=2)

ax.set_xlabel('r')
ax.set_ylabel('B(r)')
ax.set_title('Expansion Factor vs $R_{ext}$')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Internal Volume

The proper volume inside the pocket is much larger than the coordinate volume:

In [ ]:
print("Volume comparison:")
print("="*50)

for B_int in [2, 10, 100, 1000]:
    metric = VanDenBroeckMetric(R_ext=0.1, R_int=1.0, B_int=B_int)
    
    # Internal proper volume (approximate)
    V_proper = metric.internal_volume()
    
    # External coordinate volume
    V_coord = (4/3) * np.pi * metric.R_ext**3
    
    ratio = V_proper / V_coord
    
    print(f"B_int = {B_int:4d}: V_proper/V_coord ~ {ratio:.1e}")

## 4. Energy Reduction

The key advantage: energy scales with $R_{ext}^2$, not internal volume.

In [ ]:
# Compare Alcubierre and Van Den Broeck
print("Energy comparison (same internal space):")
print("="*60)

# For a 1 km internal radius
R_int = 1.0  # 1 unit in our geometric units

# Alcubierre with R=1
alc = AlcubierreMetric(v0=1.0, R=R_int)
E_alc = alc.total_energy_estimate()

print(f"Alcubierre (R={R_int}): E ~ {E_alc:.2e}")

for R_ext in [0.1, 0.01, 0.001]:
    vdb = VanDenBroeckMetric(R_ext=R_ext, R_int=R_int, B_int=R_int/R_ext)
    factor = vdb.energy_reduction_factor()
    E_vdb = E_alc * factor
    print(f"Van Den Broeck (R_ext={R_ext}): E ~ {E_vdb:.2e} (reduction: {factor:.2e})")

## 5. Energy Density Comparison

In [ ]:
# Compare energy density distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Standard Alcubierre
alc = AlcubierreMetric(v0=1.0, R=1.0)
plot_energy_density(alc, x_range=(-2, 2), y_range=(-2, 2), nx=80, ny=80, ax=axes[0])
axes[0].set_title('Alcubierre (R=1.0)')

# Van Den Broeck with small external radius
vdb = VanDenBroeckMetric(v0=1.0, R_ext=0.2, R_int=1.0, B_int=5.0)
plot_energy_density(vdb, x_range=(-0.5, 0.5), y_range=(-0.5, 0.5), nx=80, ny=80, ax=axes[1])
axes[1].set_title('Van Den Broeck (R_ext=0.2)')

plt.tight_layout()
plt.show()

## 6. The Metric Tensor

In [ ]:
vdb = VanDenBroeckMetric(v0=1.0, R_ext=0.1, R_int=1.0, B_int=10.0)

# Metric at different radii
print("Metric tensor at different radii:")
print("="*50)

for r in [0.0, 0.05, 0.1, 0.2]:
    g = vdb.metric(0, r, 0, 0)
    B = vdb.B_function(r)
    print(f"\nr = {r}: B(r) = {B:.2f}")
    print(f"g_00 = {g[0,0]:.4f}")
    print(f"g_11 = {g[1,1]:.4f}")
    print(f"g_22 = {g[2,2]:.4f}")
    print(f"g_01 = {g[0,1]:.4f}")

## 7. Energy Conditions

In [ ]:
vdb = VanDenBroeckMetric(v0=1.0, R_ext=0.1, R_int=1.0, B_int=10.0)
metric_func = vdb.get_metric_func()

# Check in different regions
test_points = [
    ('Center', [0, 0, 0, 0]),
    ('Inner region', [0, 0.03, 0.01, 0]),
    ('Transition', [0, 0.1, 0.02, 0]),
    ('Outer region', [0, 0.2, 0.05, 0]),
]

print("Energy Conditions:")
print("="*60)

for name, coords in test_points:
    coords = np.array(coords, dtype=float)
    conditions = check_energy_conditions(metric_func, coords)
    
    print(f"\n{name}:")
    for cond_name, (satisfied, value) in conditions.items():
        status = "OK" if satisfied else "VIOLATED"
        print(f"  {cond_name}: {status:8s} ({value:+.2e})")

## 8. Practical Implications

The Van Den Broeck modification shows that:

1. **Energy requirements can be reduced** by geometric modifications
2. **Large internal volumes** don't require proportionally more energy
3. **The exterior appearance** can be arbitrarily small

However, exotic matter is still required - the modification reduces the *amount*, not the *need*.

In [ ]:
# Summary table
print("Energy Requirements Summary:")
print("="*70)
print(f"{'Configuration':<30} {'Internal Vol':<15} {'Energy':<15} {'Reduction':<10}")
print("-"*70)

# Reference: Alcubierre with R=1
alc = AlcubierreMetric(v0=1.0, R=1.0)
E_ref = alc.total_energy_estimate()
V_ref = (4/3) * np.pi * 1.0**3
print(f"{'Alcubierre R=1':<30} {V_ref:<15.2f} {E_ref:<15.2e} {'1.0':<10}")

# Van Den Broeck configurations
for R_ext in [0.1, 0.01, 0.001]:
    vdb = VanDenBroeckMetric(R_ext=R_ext, R_int=1.0, B_int=1.0/R_ext)
    V_int = vdb.internal_volume()
    factor = vdb.energy_reduction_factor()
    E_vdb = E_ref * factor
    name = f"VdB R_ext={R_ext}"
    print(f"{name:<30} {V_int:<15.2f} {E_vdb:<15.2e} {factor:<10.2e}")

## Summary

The Van Den Broeck modification:

1. **Reduces energy requirements** by making the external bubble microscopic
2. **Maintains large internal volume** through expansion factor $B(r)$
3. **Energy scales as $R_{ext}^2$** rather than internal volume
4. **Still requires exotic matter** - the fundamental issue remains
5. **Demonstrates geometric creativity** - different metric constructions can have different physical properties

This was an important step in warp drive research, showing that energy requirements are not as fixed as originally thought.